<h1>Analyse lexicale</h1>

Voici une description simple pour votre cas :

- **Constantes** : dans ce cas (nombres).
- **Opérateurs** : `+`, `-`, `*`, `/`.
- **Délimiteurs** : `(`, `)` (parenthèses ouvrantes et fermantes).
- **Espaces blancs** : ignorés.

In [10]:
import re

# === Analyseur lexical ===
class MathLexer:
    rules = [
        ("NUMBER", r"\d+"),                 # Constante entière
        ("OP_ADD", r"\+"),                  # Opérateur d'addition
        ("OP_SUB", r"-"),                   # Opérateur de soustraction
        ("OP_MUL", r"\*"),                  # Opérateur de multiplication
        ("OP_DIV", r"/"),                   # Opérateur de division
        ("LPAREN", r"\("),                  # Parenthèse ouvrante
        ("RPAREN", r"\)"),                  # Parenthèse fermante
        ("WHITESPACE", r"[ \t]+"),          # Espaces (à ignorer)
    ]

    def __init__(self):
        self.tokens = []

    def tokenize(self, text):
        position = 0
        while position < len(text):
            match = None
            for token_name, token_regex in self.rules:
                regex = re.compile(token_regex)
                match = regex.match(text, position)
                if match:
                    value = match.group(0)
                    if token_name != "WHITESPACE":  # On ignore les espaces
                        self.tokens.append((token_name, value))
                    position += len(value)
                    break
            if not match:
                raise ValueError(f"Erreur lexicale : caractère inattendu '{text[position]}'")
        return self.tokens




<h1>Analyse syntaxique</h1>

L'analyse syntaxique vérifie si l'expression respecte les règles de grammaire définies :

**Expressions valides :**
Somme ou différence entre termes : a + b, a - b.
Produit ou division entre facteurs : a * b, a / b.
Parenthèses imbriquées : (a + b) * c.

**Termes** : Combinaison de constantes et d'opérateurs arithmétiques (+, -, *, /).
**Facteurs** : Constantes ou expressions entourées par des parenthèses.
**Erreurs détectées :**
Parenthèses mal appariées : (a + b.
Opérateurs sans opérandes : c +.


In [20]:
# === Analyseur syntaxique ===
class MathParser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_index = 0

    def current_token(self):
        if self.current_index < len(self.tokens):
            return self.tokens[self.current_index]
        return None

    def advance(self):
        self.current_index += 1

    def match(self, token_type):
        token = self.current_token()
        if token and token[0] == token_type:
            self.advance()
            return token
        return None

    def parse(self):
        result = self.parse_expression()
        if self.current_token():
            raise ValueError("Erreur syntaxique : tokens restants après l'analyse.")
        return result

    def parse_expression(self):
        # Règle : <EXP> ::= <TERM> ((+|-) <TERM>)*
        result = self.parse_term()
        while self.current_token() and self.current_token()[0] in {"OP_ADD", "OP_SUB"}:
            operator = self.match("OP_ADD") or self.match("OP_SUB")
            right = self.parse_term()
            if operator[0] == "OP_ADD":
                result += right
            elif operator[0] == "OP_SUB":
                result -= right
        return result

    def parse_term(self):
        # Règle : <TERM> ::= <FACTOR> ((*|/) <FACTOR>)*
        result = self.parse_factor()
        while self.current_token() and self.current_token()[0] in {"OP_MUL", "OP_DIV"}:
            operator = self.match("OP_MUL") or self.match("OP_DIV")
            right = self.parse_factor()
            if operator[0] == "OP_MUL":
                result *= right
            elif operator[0] == "OP_DIV":
                if right == 0:
                    raise ValueError("Erreur sémantique : division par zéro.")
                result /= right
        return result

    def parse_factor(self):
        # Règle : <FACTOR> ::= NUMBER | (LPAREN EXP RPAREN)
        if self.match("LPAREN"):
            result = self.parse_expression()
            if not self.match("RPAREN"):
                raise ValueError("Erreur syntaxique : parenthèse fermante manquante.")
            return result
        token = self.match("NUMBER")
        if token:
            return int(token[1])
        raise ValueError("Erreur syntaxique : facteur attendu.")


<h1>Analyse sémantique</h1>

L'analyse sémantique vérifie la validité logique de l'expression :

Division par zéro : Erreur si un opérateur / a une opérande droite égale à 0.
Exemple : 10 / 0.
Résultat attendu : Retourne un nombre valide après évaluation.

In [12]:
# === Analyseur sémantique ===
class MathSemanticAnalyzer:
    def __init__(self, result):
        self.result = result

    def analyze(self):
        # Par exemple, vérifier si le résultat dépasse certaines limites
        if self.result > 1e6:
            raise ValueError("Erreur sémantique : le résultat dépasse la limite autorisée.")
        return self.result


<h1>Application du code créé </h1>

In [18]:
# === Exemple d'utilisation ===
input_expression = "5 + (6 * 2) - (3-8) "  # Notez qu'il y a une division par zéro pour tester l'analyse sémantique

# Analyse lexicale
lexer = MathLexer()
tokens = lexer.tokenize(input_expression)
print("Tokens lexicaux extraits :", tokens)

# Analyse syntaxique
parser = MathParser(tokens)
try:
    result = parser.parse()

    # Analyse sémantique
    semantic_analyzer = MathSemanticAnalyzer(result)
    final_result = semantic_analyzer.analyze()
    print("Résultat final après analyse sémantique :", final_result)

except ValueError as e:
    print(f"Erreur : {e}")

Tokens lexicaux extraits : [('NUMBER', '5'), ('OP_ADD', '+'), ('LPAREN', '('), ('NUMBER', '6'), ('OP_MUL', '*'), ('NUMBER', '2'), ('RPAREN', ')'), ('OP_SUB', '-'), ('LPAREN', '('), ('NUMBER', '3'), ('OP_SUB', '-'), ('NUMBER', '8'), ('RPAREN', ')')]
Résultat final après analyse sémantique : 22
